# Prueba de detección de distancias
Esto detecta los links a ver si existen y te dice el elemento más próximo

In [2]:
import math

def calculate_distance(x1, y1, x2, y2):
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

def main(file_path):
    elements = []
    links = []

    # Leer el archivo
    with open(file_path, 'r') as file:
        for line in file:
            data = line.strip().split()
            class_id = int(data[0])
            center_x = float(data[1])
            center_y = float(data[2])
            width = float(data[3])
            height = float(data[4])

            if class_id == 1:  # Identificador de Link
                links.append({'x': center_x, 'y': center_y, 'width': width, 'height': height})
            else:
                element_type = {0: 'DNS', 2: 'Router', 3: 'Switch'}[class_id]
                elements.append({'id': element_type, 'x': center_x, 'y': center_y, 'width': width, 'height': height})

    # Calcular conexiones
    connections = {}
    for i, element in enumerate(elements):
        element_id = f"{element['id']}_{i+1}"
        min_distance = float('inf')
        closest_link = None
        
        for link in links:
            distance = calculate_distance(element['x'], element['y'], link['x'], link['y'])
            if distance < min_distance:
                min_distance = distance
                closest_link = link

        connections[element_id] = {'connected_to': closest_link, 'distance': min_distance}

    # Imprimir las conexiones
    for key, value in connections.items():
        print(f"{key} is connected to a Link with distance {value['distance']}")

file_path = './1.txt'
main(file_path)


Router_1 is connected to a Link with distance 0.14056075573217436
DNS_2 is connected to a Link with distance 0.17658045274605005
DNS_3 is connected to a Link with distance 0.20606288224714325
DNS_4 is connected to a Link with distance 0.20744373067653799
DNS_5 is connected to a Link with distance 0.14825935558001055
Switch_6 is connected to a Link with distance 0.18519025806451053


# Conexión entre elementos suprimiendo links
Esto formará un esquema básico suprimiendo los links, esto corrije de forma autonoma la posibilidad de que los links no existan

In [1]:
import math

elements = []
def calculate_distance(x1, y1, x2, y2):
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

def main(file_path):

    # Leer el archivo
    with open(file_path, 'r') as file:
        for line in file:
            data = line.strip().split()
            class_id = int(data[0])
            if class_id != 1:  # Ignoramos los links
                element_type = {0: 'DNS', 2: 'Router', 3: 'Switch'}[class_id]
                center_x = float(data[1])
                center_y = float(data[2])
                elements.append({'id': element_type, 'x': center_x, 'y': center_y})

    # Función para encontrar el elemento más cercano
    def find_closest_element(current, others):
        min_distance = float('inf')
        closest_element = None
        for element in others:
            distance = calculate_distance(current['x'], current['y'], element['x'], element['y'])
            if distance < min_distance:
                min_distance = distance
                closest_element = element
        return closest_element, min_distance

    # Calcular conexiones directas
    connections = {}
    for i, element in enumerate(elements):
        current_id = f"{element['id']}_{i+1}"
        others = [e for j, e in enumerate(elements) if i != j]
        closest, distance = find_closest_element(element, others)
        if closest:
            closest_id = f"{closest['id']}_{elements.index(closest)+1}"
        connections[current_id] = {'connected_to': closest_id, 'distance': distance}

    # Imprimir las conexiones
    for key, value in connections.items():
        print(f"{key} is connected to {value['connected_to']} with distance {value['distance']}")
    
file_path = './1.txt'
main(file_path)


Router_1 is connected to Switch_6 with distance 0.2806729552949481
DNS_2 is connected to Router_1 with distance 0.33973900160270093
DNS_3 is connected to DNS_4 with distance 0.36645383081637994
DNS_4 is connected to DNS_3 with distance 0.36645383081637994
DNS_5 is connected to Router_1 with distance 0.2887971355900193
Switch_6 is connected to Router_1 with distance 0.2806729552949481


In [4]:
import json
import uuid

# Función para generar una estructura de nodo basada en los datos del elemento y su tipo
def generate_node(element_id, node_type, x, y, width, height, adapters):
    node_id = str(uuid.uuid4())
    return {
        "compute_id": "local",
        "console": 5000,
        "console_auto_start": True,
        "console_type": "telnet",
        "custom_adapters": [],
        "first_port_name": None,
        "height": height,
        "label": {
            "rotation": 0,
            "style": "font-family: TypeWriter;font-size: 10.0;font-weight: bold;fill: #000000;fill-opacity: 1.0;",
            "text": element_id,
            "x": 0,
            "y": -25
        },
        "locked": True,
        "name": element_id,
        "node_id": node_id,
        "node_type": node_type,
        "port_name_format": "Ethernet{0}",
        "port_segment_size": 0,
        "properties": {
            "adapters": adapters,
            "aux": 5001,
            "console_http_path": "/",
            "console_http_port": 80,
            "console_resolution": "800x600",
            "container_id": str(uuid.uuid4()),
            "environment": f"MAC=0\nTYPE={adapters}",
            "extra_hosts": None,
            "extra_volumes": [],
            "image": "jcfabero/server:latest",
            "start_command": "/bin/bash",
            "usage": "Configure network settings."
        },
        "symbol": ":/symbols/classic/server.svg",
        "template_id": str(uuid.uuid4()),
        "width": width,
        "x": x,
        "y": y,
        "z": 1
    }

# Definir datos generales del proyecto
project_data = {
    "auto_close": True,
    "auto_open": False,
    "auto_start": False,
    "drawing_grid_size": 25,
    "grid_size": 75,
    "name": "1",
    "project_id": str(uuid.uuid4()),
    "revision": 9,
    "scene_height": 1000,
    "scene_width": 2000,
    "show_grid": False,
    "show_interface_labels": False,
    "show_layers": False,
    "snap_to_grid": False,
    "supplier": None,
    "topology": {
        "computes": [],
        "drawings": [],
        "links": [],
        "nodes": []
    },
    "type": "topology",
    "variables": None,
    "version": "2.2.29",
    "zoom": 100
}

# Añadir nodos basados en los datos de conexión
element_types = {'DNS': 'server', 'Router': 'router', 'Switch': 'switch'}
for element in elements:
    element_id = f"{element['id']}_{elements.index(element)+1}"
    node_type = element_types[element['id']]
    x = int(element['x'] * project_data['scene_width']) - 500
    y = int(element['y'] * project_data['scene_height']) - 500
    node_data = generate_node(element_id, node_type, x, y, 70, 45, 1)
    project_data['topology']['nodes'].append(node_data)

# Generar archivo JSON para el archivo .gns3
json_data = json.dumps(project_data, indent=4)

# Guardar el archivo JSON
output_file_path = './generated_topology.gns3'
with open(output_file_path, 'w') as file:
    file.write(json_data)

output_file_path


'./generated_topology.gns3'

PABLO

- Un solo link de conexión

In [22]:
import math

def calculate_distance(x1, y1, x2, y2):
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

# Encuentra el elemento más cercano en un solo bucle
def find_closest_element(element, elements):
    min_distance = float('inf')
    closest_element = None

    for other in elements:
        if other['id'] == element['id']:
            continue  # No comparar con el mismo elemento

        distance = calculate_distance(element['x'], element['y'], other['x'], other['y'])
        if distance < min_distance:
            min_distance = distance
            closest_element = other

    return closest_element, min_distance

def obtener_conexiones_validas(conexiones, links):
    # Creamos un conjunto vacío para almacenar las conexiones válidas
    conexiones_validas = set()
    # Iteramos sobre cada conexión
    for nodo, info in conexiones.items():
        nodo_a = nodo
        nodo_b = info['connected_to']

        # Verificamos si la conexión inversa existe
        if conexiones[nodo_b]['connected_to'] == nodo_a:
            # Verificamos si ambos nodos están conectados al mismo link
            link_a = links[nodo_a]['connected_to']
            link_b = links[nodo_b]['connected_to']

            if link_a == link_b:
                # Comprobamos si la conexión ya existe en el conjunto
                if (nodo_a, nodo_b) not in conexiones_validas and (nodo_b, nodo_a) not in conexiones_validas:
                    # Agregamos la conexión al conjunto
                    conexiones_validas.add((nodo_a, nodo_b))

    return conexiones_validas

def main(file_path):
    elements = []
    links = []

    # Leer el archivo
    with open(file_path, 'r') as file:
        for i, line in enumerate(file):
            data = line.strip().split()
            class_id = int(data[0])
            center_x = float(data[1])
            center_y = float(data[2])
            width = float(data[3])
            height = float(data[4])

            if class_id == 1: 
                links.append({'id': f"Link_{i+1}", 'x': center_x, 'y': center_y, 'width': width, 'height': height})
            else:
                element_type = {0: 'DNS', 2: 'Router', 3: 'Switch'}[class_id]
                elements.append({'id': f"{element_type}_{i+1}", 'x': center_x, 'y': center_y, 'width': width, 'height': height})

    # Calcular conexiones a elementos
    connections_to_elements = {}
    for element in elements:
        closest, distance = find_closest_element(element, elements)
        if closest:
            connections_to_elements[element['id']] = {
                'connected_to': closest['id'],
                'distance': distance,
            }

    # Calcular conexiones a links
    connections_to_links = {}
    for element in elements:
        closest_link = None
        min_distance = float('inf')
        
        for link in links:
            distance = calculate_distance(element['x'], element['y'], link['x'], link['y'])
            if distance < min_distance:
                min_distance = distance
                closest_link = link['id']
                
        connections_to_links[element['id']] = {
            'connected_to': closest_link,
            'distance': min_distance
        }

    # Imprimir las conexiones
    print('Conexiones entre elementos: ')
    for key, value in connections_to_elements.items():
        print(f"{key} -> {value['connected_to']}")

    print('\nConexiones entre elementos y links:')
    for key, value in connections_to_links.items():
        print(f"{key} -> {value['connected_to']}")
            
    # Obtenemos las conexiones válidas
    conexiones_validas = obtener_conexiones_validas(connections_to_elements, connections_to_links)
    
    print("Conexiones válidas:")
    for conexion in conexiones_validas:
        print(f"{conexion[0]} <-> {conexion[1]}")


file_path = './2.txt'
main(file_path)


Conexiones entre elementos: 
Router_3 -> Switch_11
Router_6 -> Switch_11
DNS_7 -> Router_6
DNS_8 -> DNS_9
DNS_9 -> Router_3
DNS_10 -> Router_6
Switch_11 -> Router_6
DNS_12 -> Switch_11

Conexiones entre elementos y links:
Router_3 -> Link_15
Router_6 -> Link_13
DNS_7 -> Link_5
DNS_8 -> Link_1
DNS_9 -> Link_4
DNS_10 -> Link_2
Switch_11 -> Link_13
DNS_12 -> Link_14

-----------------------------------------------------
Conexiones válidas:
Router_6 <-> Switch_11
-----------------------------------------------------


- Varios links de conexión

In [6]:
# import math

# def calculate_distance(x1, y1, x2, y2):
#     return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

# # Función para encontrar elementos dentro del umbral de distancia
# def find_elements_within_threshold(current, others, threshold):
#     nearby_elements = []
#     for element in others:
#         distance = calculate_distance(current['x'], current['y'], element['x'], element['y'])
#         if distance <= threshold:
#             nearby_elements.append((element, distance))
#     return nearby_elements

# def main(file_path, distance_threshold):
#     elements = []
#     links = []

#     # Leer el archivo
#     with open(file_path, 'r') as file:
#         for i, line in enumerate(file):
#             data = line.strip().split()
#             class_id = int(data[0])
#             center_x = float(data[1])
#             center_y = float(data[2])
#             width = float(data[3])
#             height = float(data[4])

#             if class_id == 1:  # Identificador de Link
#                 links.append({'id': f"Link_{i+1}", 'x': center_x, 'y': center_y, 'width': width, 'height': height})
#             else:
#                 element_type = {0: 'DNS', 2: 'Router', 3: 'Switch'}[class_id]
#                 elements.append({'id': f"{element_type}_{i+1}", 'x': center_x, 'y': center_y, 'width': width, 'height': height})

#     # Conexiones entre elementos dentro del umbral
#     connections_between_elements = {}
#     for i, element in enumerate(elements):
#         current_id = f"{element['id']}"
#         others = [e for j, e in enumerate(elements) if i != j]
#         nearby_elements = find_elements_within_threshold(element, others, distance_threshold)
#         connections_between_elements[current_id] = [(f"{e[0]['id']}", e[1]) for e in nearby_elements]

#     # Conexiones de elementos a enlaces
#     connections_to_links = {}
#     for i, element in enumerate(elements):
#         element_id = f"{element['id']}"
#         min_distance = float('inf')
#         closest_link = None
        
#         for link in links:
#             distance = calculate_distance(element['x'], element['y'], link['x'], link['y'])
#             if distance < min_distance:
#                 min_distance = distance
#                 closest_link = f"{link['id']}"

#         connections_to_links[element_id] = {'connected_to': closest_link, 'distance': min_distance}

#     # Imprimir las conexiones entre elementos dentro del umbral
#     print('Conexiones entre elementos dentro del umbral:')
#     for key, values in connections_between_elements.items():
#         connections = ', '.join([f"{v[0]} ({v[1]:.2f}m)" for v in values])
#         print(f"{key} -> {connections}")

#     # Imprimir las conexiones entre elementos y enlaces
#     print('\nConexiones entre elementos y enlaces:')
#     for key, value in connections_to_links.items():
#         print(f"{key} -> {value['connected_to']} ({value['distance']:.2f}m)")

# file_path = './2.txt'
# distance_threshold = 0.35  # Define un umbral de distancia
# main(file_path, distance_threshold)


In [62]:
import math

def calculate_distance(x1, y1, x2, y2):
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
   
def get_valid_connections(nearest_elements, nearest_links):
    conexiones_validas = set()

    # Crear un mapa de enlaces para cada elemento
    element_links = {e[0]: set([l[0] for l in e[1]]) for e in nearest_links}

    # Verificar conexiones entre elementos que compartan enlaces
    for current_element, connected_elements in nearest_elements:
        current_links = element_links.get(current_element, set())

        for connected_element, _ in connected_elements:
            # Buscar los enlaces del elemento conectado
            connected_links = element_links.get(connected_element, set())

            # Verificar si comparten al menos un enlace
            common_links = current_links.intersection(connected_links)

            if common_links:
                # Ordenar los elementos para evitar duplicados
                connection = tuple(sorted([current_element, connected_element]))
                conexiones_validas.add(connection)  # Conjunto garantiza unicidad

    return list(conexiones_validas)

# Encuentra el elemento más cercano en un solo bucle
def find_valid_element(element, elements, threshold):
    valid_elements = []
    for other in elements:
        if other['id'] == element['id']:
            continue
        distance = calculate_distance(element['x'], element['y'], other['x'], other['y'])
        if distance <= threshold:
            valid_elements.append((other['id'], distance))
    return valid_elements

def main(file_path):
    "Calculamos la distancia entre el primer elemento del txt con el resto de elementos"
    elements = []
    links = []

    # Leer el archivo
    with open(file_path, 'r') as file:
        for i, line in enumerate(file):
            data = line.strip().split()
            class_id = int(data[0])
            center_x = float(data[1])
            center_y = float(data[2])
            width = float(data[3])
            height = float(data[4])

            if class_id == 1:
                links.append({'id': f"Link_{i+1}", 'x': center_x, 'y': center_y, 'width': width, 'height': height})
            else:
                element_type = {0: 'DNS', 2: 'Router', 3: 'Switch'}[class_id]
                elements.append({'id': f"{element_type}_{i+1}", 'x': center_x, 'y': center_y, 'width': width, 'height': height})

    # Calcular elementos cercanos
    nearest_elements = []
    for element in elements:
        valid_elements = find_valid_element(element, elements, 0.35)
        nearest_elements.append((element['id'], valid_elements))
            
    # Calcular links cercanos
    nearest_links = []
    for element in elements:
        valid_links = find_valid_element(element, links, 0.15)
        nearest_links.append((element['id'], valid_links))

    # Obtenemos las conexiones válidas        
    connections= get_valid_connections(nearest_elements, nearest_links)
    
    #  Pintamos
    print('Conexiones entre elementos:')
    for element, others in nearest_elements:
        for other, distance in others:
            print(f"{element} -> {other} ({distance:.2f}m)")
            
    print('\nConexiones entre elementos y links:')
    for element, links in nearest_links:
        for link, distance in links:
            print(f"{element} -> {link} ({distance:.2f}m)")
    
    print('\n---------------------------------')
    print('Conexiones válidas:')
    for connection in connections:
        print(f"{connection[0]} <-> {connection[1]}")
    print('---------------------------------')

file_path = './2.txt'
main(file_path)

Conexiones entre elementos:
Router_3 -> DNS_8 (0.29m)
Router_3 -> DNS_9 (0.28m)
Router_3 -> Switch_11 (0.22m)
Router_6 -> DNS_7 (0.22m)
Router_6 -> DNS_10 (0.28m)
Router_6 -> Switch_11 (0.20m)
DNS_7 -> Router_6 (0.22m)
DNS_8 -> Router_3 (0.29m)
DNS_8 -> DNS_9 (0.29m)
DNS_9 -> Router_3 (0.28m)
DNS_9 -> DNS_8 (0.29m)
DNS_10 -> Router_6 (0.28m)
Switch_11 -> Router_3 (0.22m)
Switch_11 -> Router_6 (0.20m)
Switch_11 -> DNS_12 (0.32m)
DNS_12 -> Switch_11 (0.32m)

Conexiones entre elementos y links:
Router_3 -> Link_4 (0.14m)
Router_3 -> Link_15 (0.11m)
Router_6 -> Link_5 (0.12m)
Router_6 -> Link_13 (0.10m)
DNS_7 -> Link_5 (0.11m)
DNS_8 -> Link_1 (0.14m)
DNS_9 -> Link_4 (0.14m)
DNS_10 -> Link_2 (0.12m)
Switch_11 -> Link_13 (0.10m)
Switch_11 -> Link_14 (0.14m)
Switch_11 -> Link_15 (0.11m)
Switch_11 -> Link_16 (0.14m)

---------------------------------
Conexiones válidas:
DNS_9 <-> Router_3
DNS_7 <-> Router_6
Router_3 <-> Switch_11
Router_6 <-> Switch_11
---------------------------------
